# 📊 Kasparro — Applied AI Market Intelligence (Kaggle Notebook)

**Author:** Viresh Nagouda

This notebook:
- Cleans & unifies **Google Play (Kaggle)** + **Apple App Store (iTunes/RapidAPI)** data
- Ranks categories by **opportunity** + **confidence**
- Generates **executive recommendations** (Gemini) with robust prompting
- Exports **combined_apps.csv**, **insights.json**, **top_categories.png**, **report.html**
- Includes a **resilient Streamlit app**
- Phase 5: **D2C funnel + SEO + creatives**

**Run order:** top → bottom.


In [32]:
import os, json, math, time, textwrap
from typing import Optional, Dict, Any
import numpy as np
import pandas as pd

# Detect Kaggle output directory
OUTPUT_DIR = "/kaggle/working" if os.path.exists("/kaggle/working") else "out"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Paths for Kaggle vs local
KAGGLE_ANDROID_PATH = "/kaggle/input/google-play-store-apps/googleplaystore.csv"
LOCAL_ANDROID_PATH  = "/kaggle/input/google-play-store-apps/googleplaystore.csv"
ANDROID_PATH = KAGGLE_ANDROID_PATH if os.path.exists(KAGGLE_ANDROID_PATH) else LOCAL_ANDROID_PATH

# Optional Phase 5 Excel (attach via Add Data if available)
PHASE5_XLSX_CANDIDATES = [
    "/kaggle/input/kasparro-phase5-d2c-synthetic-dataset/Kasparro_Phase5_D2C_Synthetic_Dataset.xlsx"
]
PHASE5_XLSX_PATH = next((p for p in PHASE5_XLSX_CANDIDATES if os.path.exists(p)), None)

print("OUTPUT_DIR:", OUTPUT_DIR)
print("Phase 5 Excel found:", bool(PHASE5_XLSX_PATH))


OUTPUT_DIR: /kaggle/working
Phase 5 Excel found: True


In [33]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

GEMINI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")  # Google AI Studio key
RAPIDAPI_KEY   = user_secrets.get_secret("RAPIDAPI_KEY")    # Optional

print("Gemini key available:", bool(GEMINI_API_KEY))
print("RapidAPI key available:", bool(RAPIDAPI_KEY))


Gemini key available: True
RapidAPI key available: True


In [34]:
# Safe to run even if already installed; will be quick or no-op offline.
try:
    import google.generativeai as genai
except:
    try:
        !pip -q install google-generativeai
        import google.generativeai as genai
    except Exception as e:
        print("Could not install google-generativeai (likely internet disabled). Using fallbacks.", e)


In [35]:
if not os.path.exists(ANDROID_PATH):
    raise FileNotFoundError("Google Play dataset not found. Attach `lava18/google-play-store-apps` via Add Data.")

raw_gp = pd.read_csv(ANDROID_PATH)
print("Raw Google Play shape:", raw_gp.shape)
raw_gp.head()


Raw Google Play shape: (10841, 13)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [36]:
def parse_price(x):
    if pd.isna(x): return 0.0
    s = str(x).strip().replace("$","")
    try: return float(s)
    except: return 0.0

def parse_installs(x):
    if pd.isna(x): return np.nan
    s = str(x).replace("+","").replace(",","").strip()
    try: return int(s)
    except: return np.nan

def parse_size(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().upper()
    mult = 1.0
    if s.endswith("M"): mult = 1_000_000; s = s[:-1]
    elif s.endswith("K"): mult = 1_000;    s = s[:-1]
    s = s.replace(",","")
    try: return float(s)*mult/(1024*1024)
    except: return np.nan

def normalize_category(cat):
    if pd.isna(cat): return "Unknown"
    c = str(cat).strip().title().replace("&"," & ")
    return " ".join(c.split())

def clean_google_play(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df["App"].notna() & df["Category"].notna()].copy()
    df = df.drop_duplicates(subset=["App","Category","Reviews","Installs"], keep="first")
    df["price_usd"]         = df["Price"].apply(parse_price)
    df["installs"]          = df["Installs"].apply(parse_installs)
    df["size_mb"]           = df["Size"].apply(parse_size)
    df["rating"]            = pd.to_numeric(df["Rating"], errors="coerce")
    df["reviews_count"]     = pd.to_numeric(df["Reviews"], errors="coerce")
    df["primary_category"]  = df["Category"].apply(normalize_category)
    df["app_name"]          = df["App"].astype(str).str.strip()
    df["content_rating"]    = df["Content Rating"].astype(str).str.strip()
    df["last_updated"]      = pd.to_datetime(df["Last Updated"], errors="coerce")
    df["platform"]          = "android"
    cols = ["app_name","platform","primary_category","rating","reviews_count",
            "price_usd","installs","size_mb","content_rating","last_updated"]
    return df[cols].reset_index(drop=True)

gp = clean_google_play(raw_gp)
print("Cleaned Google Play:", gp.shape)
gp.head()


Cleaned Google Play: (10356, 10)


,app_name,platform,primary_category,rating,reviews_count,price_usd,installs,size_mb,content_rating,last_updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,android,Art_And_Design,4.1,159.0,0.0,10000.0,18.119812,Everyone,2018-01-07
1,Coloring book moana,android,Art_And_Design,3.9,967.0,0.0,500000.0,13.351440,Everyone,2018-01-15
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",android,Art_And_Design,4.7,87510.0,0.0,5000000.0,8.296967,Everyone,2018-08-01
3,Sketch - Draw & Paint,android,Art_And_Design,4.5,215644.0,0.0,50000000.0,23.841858,Teen,2018-06-08
4,Pixel Draw - Number Art Coloring Book,android,Art_And_Design,4.3,967.0,0.0,100000.0,2.670288,Everyone,2018-06-20


In [39]:
import requests
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


def _itunes_search(term: str, country: str="us", limit:int=1):
    url = "https://itunes.apple.com/search"
    params = {"term": term, "country": country, "entity": "software", "limit": limit}
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r.json()

APPSTORE_HOST = "appstore-scrapper-api.p.rapidapi.com"

def _rapidapi_search(term: str, country: str="us"):
    if not RAPIDAPI_KEY:
        raise RuntimeError("No RapidAPI key")
    url = f"https://{APPSTORE_HOST}/search"
    headers = {"x-rapidapi-key": RAPIDAPI_KEY, "x-rapidapi-host": APPSTORE_HOST}
    params = {"term": term, "country": country, "lang": "en_us"}
    r = requests.get(url, headers=headers, params=params, timeout=15)
    r.raise_for_status()
    return r.json()

def appstore_search(term: str, country: str="us") -> Dict[str, Any]:
    # 1) iTunes (no key)
    try:
        data = _itunes_search(term, country, 1)
        results = data.get("results", [])
        if results:
            r = results[0]
            return {"results": [{
                "trackName": r.get("trackName"),
                "primaryGenreName": r.get("primaryGenreName"),
                "averageUserRating": r.get("averageUserRating"),
                "userRatingCount": r.get("userRatingCount"),
                "price": r.get("price", 0.0),
                "isFree": (r.get("price", 0.0) in [0, 0.0]),
                "contentAdvisoryRating": r.get("contentAdvisoryRating"),
                "currentVersionReleaseDate": r.get("currentVersionReleaseDate"),
                "fileSizeBytes": r.get("fileSizeBytes"),
            }]}
    except Exception as e:
        print("iTunes Search API failed → trying RapidAPI:", e)

    # 2) RapidAPI
    try:
        return _rapidapi_search(term, country)
    except Exception as e:
        print("RapidAPI error → using mock:", e)

    # 3) Mock
    return {"results": [{
        "trackName": f"{term.title()} App (Mock)",
        "primaryGenreName": "Utilities",
        "averageUserRating": 4.2,
        "userRatingCount": 5432,
        "price": 0.99,
        "isFree": False,
        "contentAdvisoryRating": "9+",
        "currentVersionReleaseDate": "2024-02-10T10:00:00Z",
        "fileSizeBytes": str(80*1024*1024),
    }]}

def transform_appstore_result(r: Dict[str, Any]) -> Dict[str, Any]:
    try:
        size_mb = float(r.get("fileSizeBytes", 0)) / (1024*1024) if r.get("fileSizeBytes") else None
    except: size_mb = None
    return {
        "app_name":         r.get("trackName"),
        "platform":         "ios",
        "primary_category": r.get("primaryGenreName"),
        "rating":           r.get("averageUserRating"),
        "reviews_count":    r.get("userRatingCount"),
        "price_usd":        float(r.get("price", 0.0)) if r.get("price") is not None else 0.0,
        "installs":         np.nan,  # iOS installs unavailable
        "size_mb":          size_mb,
        "content_rating":   r.get("contentAdvisoryRating"),
        "last_updated":     pd.to_datetime(r.get("currentVersionReleaseDate"), errors="coerce"),
    }

sample_terms = ["notion","slack","spotify"]
ios_rows = []
for t in sample_terms:
    data = appstore_search(t)
    for r in data.get("results", [])[:1]:
        ios_rows.append(transform_appstore_result(r))
ios_df = pd.DataFrame(ios_rows)
print("iOS sample rows:", ios_df.shape)
ios_df.head()


iOS sample rows: (3, 10)


,app_name,platform,primary_category,rating,reviews_count,price_usd,installs,size_mb,content_rating,last_updated
0,"Notion: Notes, Tasks, AI",ios,Productivity,4.78651,67496,0.0,NaN,310.940430,4+,2025-09-23 16:54:53+00:00
1,Slack,ios,Business,3.89668,29626,0.0,NaN,398.038086,17+,2025-09-26 15:01:08+00:00
2,Spotify: Music and Podcasts,ios,Music,4.78773,37059921,0.0,NaN,220.545898,12+,2025-09-24 14:06:51+00:00


In [40]:
combined = pd.concat([gp, ios_df], ignore_index=True)

# Drop garbage categories like "1.9" (no letters / too short)
mask_valid_cat = combined["primary_category"].astype(str).str.contains(r"[A-Za-z]", na=False)
combined_clean = combined[mask_valid_cat & (combined["primary_category"].str.len() >= 2)].copy()

# Sanity checks
assert combined_clean["app_name"].notna().mean() > 0.95
assert combined_clean["primary_category"].notna().mean() > 0.90

combined_path = os.path.join(OUTPUT_DIR, "combined_apps.csv")
combined_clean.to_csv(combined_path, index=False)
combined_path


'/kaggle/working/combined_apps.csv'

In [41]:
# 1) Use only Android rows for installs (iOS lacks installs)
android_only = combined_clean[combined_clean["platform"] == "android"].copy()

# 2) Core stats across both platforms
core_stats = combined_clean.groupby("primary_category").agg(
    n_apps=("app_name","count"),
    avg_rating=("rating","mean"),
    sum_reviews=("reviews_count","sum"),
).reset_index()

# 3) Median installs from Android only
med_installs_android = (
    android_only
    .groupby("primary_category")["installs"]
    .median()
    .rename("med_installs")
    .reset_index()
)

# 4) Merge + fill NaN medians to 0 (unknown)
cat_stats = core_stats.merge(med_installs_android, on="primary_category", how="left")
cat_stats["med_installs"] = pd.to_numeric(cat_stats["med_installs"], errors="coerce").fillna(0)

def zscore(series: pd.Series):
    return (series - series.mean()) / (series.std(ddof=0) + 1e-9)

cat_stats["z_installs"] = zscore(np.log1p(cat_stats["med_installs"]))
cat_stats["z_rating"]   = zscore(cat_stats["avg_rating"].fillna(cat_stats["avg_rating"].median()))
cat_stats["z_comp"]     = zscore(-cat_stats["n_apps"])  # fewer apps → better

cat_stats["opportunity_score"] = (
    0.5*cat_stats["z_installs"] + 0.4*cat_stats["z_rating"] + 0.1*cat_stats["z_comp"]
)

var_penalty = 1 - (cat_stats["avg_rating"].fillna(0).std() / 5.0)
var_penalty = np.clip(var_penalty, 0.6, 1.0)
cat_stats["confidence"] = np.minimum(1.0, np.sqrt(cat_stats["n_apps"]/50.0)) * var_penalty

top_cats = cat_stats.sort_values("opportunity_score", ascending=False).head(10)
top_cats


,primary_category,n_apps,avg_rating,sum_reviews,med_installs,z_installs,z_rating,z_comp,opportunity_score,confidence
8,Education,130,4.375969,2.316550e+07,1000000.0,0.870535,1.125009,0.482186,0.933490,0.971001
33,Weather,82,4.244000,1.460474e+07,1000000.0,0.870535,0.201163,0.614710,0.577204,0.971001
27,Shopping,223,4.252239,9.486276e+07,1000000.0,0.870535,0.258839,0.225421,0.561345,0.971001
28,Social,280,4.254918,5.335768e+08,1000000.0,0.870535,0.277595,0.068048,0.553110,0.971001
9,Entertainment,111,4.136036,4.757072e+07,5000000.0,1.415973,-0.554636,0.534643,0.539597,0.971001
25,Photography,322,4.182895,2.042974e+08,3000000.0,1.242854,-0.226603,-0.047910,0.525995,0.971001
0,Art_And_Design,65,4.358065,1.714440e+06,100000.0,0.090191,0.999669,0.661645,0.511128,0.971001
15,Health_And_Fitness,306,4.261450,3.084519e+07,500000.0,0.635627,0.323324,-0.003735,0.446770,0.971001
14,Game,1121,4.281285,1.415537e+09,1000000.0,0.870535,0.462176,-2.253882,0.394749,0.971001
23,Parenting,60,4.300000,9.583310e+05,100000.0,0.090191,0.593190,0.675450,0.349917,0.971001


In [43]:
# --- LLM Summary v3: Structured JSON + Few-shot + Validation ---
import json, time, re

def rows_for_context(df: pd.DataFrame):
    rows = []
    for _, r in df.iterrows():
        rows.append({
            "primary_category": str(r["primary_category"]),
            "avg_rating": float(r["avg_rating"]) if pd.notna(r["avg_rating"]) else None,
            "median_installs": int(r["med_installs"]) if pd.notna(r["med_installs"]) else 0,
            "n_apps": int(r["n_apps"]),
            "opportunity_score": float(r["opportunity_score"]),
            "confidence": float(r["confidence"]),
        })
    return rows

def build_llm_prompt_v3(top_df: pd.DataFrame) -> str:
    context = {
        "objective": "Propose 4 crisp, actionable growth moves for the best app categories.",
        "audience": "Non-technical product & growth stakeholders.",
        "metrics": {
            "opportunity_score": "0.5*z(installs) + 0.4*z(rating) + 0.1*z(-n_apps)",
            "confidence": "min(1, sqrt(n_apps/50)) * variance_penalty_on_rating_dispersion"
        },
        "confidence_bucket_rules": {
            "high": "confidence >= 0.90",
            "medium": "0.75 <= confidence < 0.90",
            "low": "confidence < 0.75"
        },
        "output_schema": {
            "recommendations": [
                {
                    "category": "string",
                    "action": "string (1 concrete move)",
                    "why_short": "string (<= 20 words, grounded in provided metrics only)",
                    "primary_kpi": "string (e.g., DAU, CTR, CR, Retention D30)",
                    "experiment": "string (1 quick test; <= 12 words)",
                    "confidence_bucket": "one of: high|medium|low"
                }
            ]
        },
        "constraints": [
            "Use only the provided category metrics; do not invent market facts.",
            "Exactly 4 recommendations.",
            "Return JSON only. No markdown, no extra text."
        ]
    }

    # FEW-SHOT with miniature example
    few_shot_input = [
        {"primary_category":"Education","avg_rating":4.38,"median_installs":1000000,"n_apps":130,"opportunity_score":0.93,"confidence":0.97},
        {"primary_category":"Weather","avg_rating":4.24,"median_installs":1000000,"n_apps":82,"opportunity_score":0.58,"confidence":0.97}
    ]
    few_shot_output = {
        "recommendations":[
            {
                "category":"Education",
                "action":"Launch syllabus onboarding + EDU bundles",
                "why_short":"High installs & ratings with solid confidence",
                "primary_kpi":"DAU",
                "experiment":"A/B test onboarding (2 weeks)",
                "confidence_bucket":"high"
            },
            {
                "category":"Weather",
                "action":"Ship alert widgets + push cadence",
                "why_short":"Broad installs; strong ratings; steady daily use",
                "primary_kpi":"DAU",
                "experiment":"Geo-based severe-alert trial",
                "confidence_bucket":"high"
            }
        ]
    }

    current_input = rows_for_context(top_df)

    prompt = f"""
You are a senior growth analyst.

CONTEXT:
{json.dumps(context, indent=2)}

FEW_SHOT_INPUT:
{json.dumps(few_shot_input, indent=2)}

FEW_SHOT_OUTPUT:
{json.dumps(few_shot_output, indent=2)}

CURRENT_INPUT:
{json.dumps(current_input, indent=2)}

TASK:
Using CURRENT_INPUT, return JSON matching the "output_schema" with exactly 4 items in "recommendations".
- Derive "confidence_bucket" from the provided rule thresholds.
- Each "action" must be one concrete move (e.g., "SEO content cluster", "widget rollout", "trial paywall", "bundled pricing").
- "why_short" must be grounded in the given numbers only (installs/ratings/n_apps/score/confidence).
- Choose a primary_kpi from: ["DAU","CTR","Retention D30","CR"]. 
- Do NOT invent external facts; stick to the input numbers.
- Return JSON only. No markdown, no extra commentary.

"""
    return prompt.strip()

def _confidence_bucket(c: float) -> str:
    if c >= 0.90: return "high"
    if c >= 0.75: return "medium"
    return "low"

def render_structured_to_bullets(struct: dict) -> str:
    """
    Convert structured recs into polished bullets:
    - KPIs are explained in plain English
    - Confidence bucket is spelled out
    """
    kpi_explainer = {
        "DAU": "Daily Active Users (how many people open the app each day)",
        "Retention D30": "Retention after 30 days (how many users are still active)",
        "CTR": "Click-Through Rate (how often users click after seeing an item)",
        "CR": "Conversion Rate (how many users complete a purchase or signup)"
    }
    out_lines = []
    recs = struct.get("recommendations", []) or []
    for r in recs:
        cat = r.get("category","Unknown")
        action = r.get("action","")
        why = r.get("why_short","")
        kpi = r.get("primary_kpi","KPI")
        kpi_expl = kpi_explainer.get(kpi, kpi)
        confb = r.get("confidence_bucket","")
        conf_str = {"high":"High confidence","medium":"Moderate confidence","low":"Low confidence"}.get(confb,confb)
        out_lines.append(
            f"• Focus **{cat}**: {action}. "
            f"Why: {why}. "
            f"KPI: {kpi} = {kpi_expl}. "
            f"Confidence: {conf_str}."
        )
    return "\n".join(out_lines) if out_lines else "(No recommendations)"

def llm_gemini_summary_v3(top_df: pd.DataFrame) -> Dict[str, str]:
    """
    Returns {"text": human_readable_bullets, "structured": json.dumps(schema)}.
    Falls back to heuristic if the model output is not valid JSON.
    """
    # Heuristic fallback if no key
    if not GEMINI_API_KEY:
        # Deterministic top-4 heuristic
        top4 = top_df.head(4).copy()
        items = []
        for _, r in top4.iterrows():
            cb = _confidence_bucket(float(r["confidence"]))
            items.append({
                "category": r["primary_category"],
                "action": "Improve onboarding & ASO",
                "why_short": "High score; solid ratings; room to grow",
                "primary_kpi": "DAU",
                "experiment": "A/B onboarding (2 weeks)",
                "confidence_bucket": cb
            })
        struct = {"recommendations": items}
        return {"text": render_structured_to_bullets(struct), "structured": json.dumps(struct, indent=2)}

    # With key → call Gemini
    try:
        import google.generativeai as genai
        genai.configure(api_key=GEMINI_API_KEY)
        model_id = "models/gemini-2.0-flash"
        prompt = build_llm_prompt_v3(top_df)

        generation_config = {
            "temperature": 0.2,
            "top_p": 0.9,
            "top_k": 40,
            "max_output_tokens": 700,
        }

        last_err = None
        for attempt in range(3):
            try:
                model = genai.GenerativeModel(model_id, generation_config=generation_config)
                resp = model.generate_content(prompt, request_options={"timeout": 60})
                raw = (getattr(resp, "text", None) or "").strip()
                # Extract first JSON object or array from the text
                # Prefer object { ... }, but handle if the model returns only the list [ ... ]
                m_obj = re.search(r"\{[\s\S]*\}", raw)
                m_arr = re.search(r"\[[\s\S]*\]", raw)
                blob = m_obj.group(0) if m_obj else (m_arr.group(0) if m_arr else "")
                data = json.loads(blob) if blob else {}
                # Normalize to {"recommendations":[...]}
                if isinstance(data, list):
                    data = {"recommendations": data}
                if "recommendations" not in data or not isinstance(data["recommendations"], list):
                    raise ValueError("No 'recommendations' array in model output.")
                # Keep exactly 4 items (trim or pad with heuristics)
                recs = data["recommendations"][:4]
                if len(recs) < 4:
                    # pad from top_df heuristically
                    for _, r in top_df.head(4 - len(recs)).iterrows():
                        recs.append({
                            "category": r["primary_category"],
                            "action": "Improve onboarding & ASO",
                            "why_short": "High score; solid ratings; room to grow",
                            "primary_kpi": "DAU",
                            "experiment": "A/B onboarding (2 weeks)",
                            "confidence_bucket": _confidence_bucket(float(r["confidence"]))
                        })
                data["recommendations"] = recs
                text = render_structured_to_bullets(data)
                return {"text": text, "structured": json.dumps(data, indent=2)}
            except Exception as e:
                last_err = e
                time.sleep(1.2*(attempt+1))
                continue
        # Final fallback (heuristic)
        top4 = top_df.head(4).copy()
        items = []
        for _, r in top4.iterrows():
            items.append({
                "category": r["primary_category"],
                "action": "Improve onboarding & ASO",
                "why_short": "High score; solid ratings; room to grow",
                "primary_kpi": "DAU",
                "experiment": "A/B onboarding (2 weeks)",
                "confidence_bucket": _confidence_bucket(float(r["confidence"]))
            })
        struct = {"recommendations": items}
        note = f"(Gemini error fallback) {last_err}"
        return {"text": note + "\n\n" + render_structured_to_bullets(struct), "structured": json.dumps(struct, indent=2)}
    except Exception as e:
        # Import/config failure fallback
        top4 = top_df.head(4).copy()
        items = []
        for _, r in top4.iterrows():
            items.append({
                "category": r["primary_category"],
                "action": "Improve onboarding & ASO",
                "why_short": "High score; solid ratings; room to grow",
                "primary_kpi": "DAU",
                "experiment": "A/B onboarding (2 weeks)",
                "confidence_bucket": _confidence_bucket(float(r["confidence"]))
            })
        struct = {"recommendations": items}
        note = f"(Gemini import/config error) {e}"
        return {"text": note + "\n\n" + render_structured_to_bullets(struct), "structured": json.dumps(struct, indent=2)}

# ---- Generate structured + text outputs
llm_out = llm_gemini_summary_v3(top_cats)
llm_summary = llm_out["text"]
llm_structured = llm_out["structured"]
print(llm_summary)


• Focus **Education**: Personalized learning path. Why: High installs, ratings, and opportunity score with great confidence.. KPI: Retention D30 = Retention after 30 days (how many users are still active). Confidence: High confidence.
• Focus **Weather**: Implement hyperlocal alerts. Why: High installs, solid ratings, and great confidence.. KPI: DAU = Daily Active Users (how many people open the app each day). Confidence: High confidence.
• Focus **Shopping**: Introduce price comparison feature. Why: High installs, good ratings, and great confidence.. KPI: CTR = Click-Through Rate (how often users click after seeing an item). Confidence: High confidence.
• Focus **Social**: Launch group-based challenges. Why: High installs, good ratings, and great confidence.. KPI: DAU = Daily Active Users (how many people open the app each day). Confidence: High confidence.


In [44]:
insights = {
    "generated_at_utc": pd.Timestamp.utcnow().isoformat(),
    "methodology": {
        "opportunity_score": "0.5*z_installs + 0.4*z_rating + 0.1*z_comp",
        "confidence": "min(1, sqrt(n_apps/50)) * variance_penalty_on_rating_dispersion"
    },
    "top_categories": top_cats.to_dict(orient="records"),
    "llm_summary": llm_summary,             # human-friendly bullets
    "llm_structured": json.loads(llm_structured)  # strict JSON schema
}
insights_path = os.path.join(OUTPUT_DIR, "insights.json")
with open(insights_path, "w", encoding="utf-8") as f:
    json.dump(insights, f, indent=2, default=str)
print("✅ insights.json (text + structured) refreshed:", insights_path)


✅ insights.json (text + structured) refreshed: /kaggle/working/insights.json


In [45]:
import matplotlib.pyplot as plt

try:
    plot_path = os.path.join(OUTPUT_DIR, "top_categories.png")
    top_plot = top_cats.sort_values("opportunity_score", ascending=True)
    plt.figure(figsize=(8,5))
    plt.barh(top_plot["primary_category"], top_plot["opportunity_score"])
    plt.title("Top Categories by Opportunity Score")
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()
    print("Chart saved:", plot_path)
except Exception as e:
    print("Chart generation failed:", e)


Chart saved: /kaggle/working/top_categories.png


In [46]:
def render_html_report(combined_df: pd.DataFrame, top_cats_df: pd.DataFrame, llm_summary: str) -> str:
    head = """
    <html><head><meta charset="utf-8"><title>Kasparro Market Intelligence Report</title>
    <style>
    body { font-family: Arial, sans-serif; margin: 24px; }
    h1,h2 { margin-top: 1.1em; }
    table { border-collapse: collapse; width: 100%; margin-top: 12px; }
    th, td { border: 1px solid #ddd; padding: 8px; font-size: 14px; }
    th { background: #f6f6f6; }
    code, pre { background: #f3f3f3; padding: 8px; display: block; white-space: pre-wrap; }
    img { max-width: 100%; height: auto; }
    .note { color: #666; font-size: 12px; }
    </style></head><body>
    """
    title = "<h1>Kasparro — AI-Powered Market Intelligence</h1>"
    meta  = f"<p><small>Generated: {pd.Timestamp.utcnow().strftime('%Y-%m-%d %H:%M UTC')}</small></p>"
    table = top_cats_df.to_html(index=False)
    sample = combined_df.sample(min(10, len(combined_df)), random_state=42).sort_index().to_html(index=False)
    body = f"""
    <h2>Recommendations</h2>
    <pre>{llm_summary}</pre>
    <h2>Top Categories (Opportunity Ranking)</h2>
    {table}
    <h2>Visualization</h2>
    <img src="top_categories.png"/>
    <p class="note">Note: iOS installs are not exposed; median installs computed from Android only.</p>
    <h2>Combined Dataset — Sample</h2>
    {sample}
    """
    return head + title + meta + body + "</body></html>"

html = render_html_report(combined_clean, top_cats, llm_summary)
report_path = os.path.join(OUTPUT_DIR, "report.html")
with open(report_path, "w", encoding="utf-8") as f:
    f.write(html)
report_path


'/kaggle/working/report.html'

In [47]:
def query_insights(category: Optional[str]=None, top_k: int=5):
    if category:
        sub = combined_clean[combined_clean["primary_category"].str.lower() == category.lower()]
        if sub.empty:
            print("No data for category:", category)
            return
        display(sub.describe(include="all"))
    else:
        display(top_cats.head(top_k)[["primary_category","opportunity_score","confidence","avg_rating","med_installs","n_apps"]])

query_insights()  # top-5


,primary_category,opportunity_score,confidence,avg_rating,med_installs,n_apps
8,Education,0.933490,0.971001,4.375969,1000000.0,130
33,Weather,0.577204,0.971001,4.244000,1000000.0,82
27,Shopping,0.561345,0.971001,4.252239,1000000.0,223
28,Social,0.553110,0.971001,4.254918,1000000.0,280
9,Entertainment,0.539597,0.971001,4.136036,5000000.0,111


In [50]:
app_py = r"""
import json, os, traceback, pandas as pd, streamlit as st
import matplotlib.pyplot as plt

st.set_page_config(page_title='Kasparro Market Intelligence', layout='wide')
st.title('📊 Kasparro — AI-Powered Market Intelligence')

# ---------- Helpers ----------
def load_json(path: str):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        st.error(f'File not found: {path}')
    except json.JSONDecodeError as e:
        st.error(f'JSON parse error in {path}: {e}')
        with st.expander("Details"):
            st.code(traceback.format_exc())
    except Exception as e:
        st.error(f'Unexpected error loading {path}: {e}')
        with st.expander("Details"):
            st.code(traceback.format_exc())
    return {}

def load_csv(path: str):
    try:
        return pd.read_csv(path)
    except FileNotFoundError:
        st.error(f'File not found: {path}')
    except Exception as e:
        st.error(f'Error loading CSV {path}: {e}')
        with st.expander("Details"):
            st.code(traceback.format_exc())
    return pd.DataFrame()

def safe_dataframe(df: pd.DataFrame, *, fallback_msg: str):
    try:
        if df is None or df.empty:
            st.info(fallback_msg)
        else:
            st.dataframe(df)
    except Exception as e:
        st.error(f'Failed to display table: {e}')
        with st.expander("Details"):
            st.code(traceback.format_exc())

def safe_image(path: str, caption: str):
    try:
        if os.path.exists(path):
            st.image(path, caption=caption, use_column_width=True)
        else:
            st.info(f'Image not found: {path}')
    except Exception as e:
        st.error(f'Failed to render image {path}: {e}')
        with st.expander("Details"):
            st.code(traceback.format_exc())

def validate_insights_schema(ins: dict) -> dict:
    
    #Return a normalized insights dict and report any issues.
   
    issues = []
    if not isinstance(ins, dict):
        issues.append("insights.json is not an object")
        ins = {}
    if "top_categories" not in ins or not isinstance(ins.get("top_categories"), list):
        issues.append("Missing or invalid 'top_categories' (expected list)")
        ins["top_categories"] = []
    if "llm_summary" not in ins or not isinstance(ins.get("llm_summary"), str):
        issues.append("Missing or invalid 'llm_summary' (expected string)")
        ins["llm_summary"] = ""
    # Optional structured block
    if "llm_structured" in ins and not isinstance(ins.get("llm_structured"), dict):
        issues.append("Invalid 'llm_structured' (expected object)")
        ins["llm_structured"] = None
    if issues:
        with st.expander("Insights schema notes"):
            for i in issues:
                st.write("•", i)
    return ins


def safe_bar_chart_top_categories(top_df: pd.DataFrame):
    try:
        if top_df is None or top_df.empty:
            st.info("No top categories available to plot.")
            return
        required = {"primary_category", "opportunity_score"}
        if not required.issubset(set(map(str, top_df.columns))):
            st.info("Plot skipped: 'primary_category' or 'opportunity_score' not found.")
            return
        # Plot
        sorted_df = top_df.sort_values("opportunity_score", ascending=True)
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.barh(sorted_df["primary_category"], sorted_df["opportunity_score"])
        ax.set_title("Top Categories by Opportunity Score")
        ax.set_xlabel("Opportunity Score")
        ax.set_ylabel("Category")
        st.pyplot(fig)
    except Exception as e:
        st.error(f"Could not render chart: {e}")
        with st.expander("Details"):
            st.code(traceback.format_exc())

# ---------- Load artifacts (guarded) ----------
ins = load_json('insights.json')
df  = load_csv('combined_apps.csv')
ins = validate_insights_schema(ins)

# ---------- Sidebar (downloads) ----------
with st.sidebar:
    st.header("Artifacts")
    for path, label in [
        ("insights.json", "insights.json"),
        ("combined_apps.csv", "combined_apps.csv"),
        ("top_categories.png", "top_categories.png"),
        ("report.html", "report.html"),
    ]:
        try:
            if os.path.exists(path):
                with open(path, "rb") as f:
                    st.download_button(label=f"Download {label}", data=f, file_name=label)
            else:
                st.caption(f"Missing: {label}")
        except Exception as e:
            st.caption(f"Download error: {label}")
            with st.expander(f"Details: {label}"):
                st.code(str(e))

# ---------- Executive Summary ----------
st.subheader('Executive Recommendations')
try:
    summary = ins.get('llm_summary') or ""
    if summary.strip():
        st.write(summary)
    else:
        st.info('No LLM summary found. Re-run the notebook to generate insights.')
except Exception as e:
    st.error(f"Failed to render executive recommendations: {e}")
    with st.expander("Details"):
        st.code(traceback.format_exc())

# ---------- Structured Recommendations (bonus) ----------
st.subheader("Structured Recommendations (JSON)")
try:
    llm_struct = ins.get("llm_structured")
    if llm_struct and isinstance(llm_struct, dict) and "recommendations" in llm_struct:
        safe_dataframe(pd.DataFrame(llm_struct["recommendations"]),
                       fallback_msg="Structured recommendations present but empty.")
    else:
        st.info("No structured recommendations found. Re-run the notebook to regenerate insights.")
except Exception as e:
    st.error(f"Failed to render structured recommendations: {e}")
    with st.expander("Details"):
        st.code(traceback.format_exc())

# ---------- Top Categories ----------
st.subheader('Top Categories by Opportunity Score')
try:
    top_cats = pd.DataFrame(ins.get('top_categories', []))
    safe_dataframe(top_cats, fallback_msg='Top categories not available in insights.json.')
except Exception as e:
    st.error(f"Failed to render top categories: {e}")
    with st.expander("Details"):
        st.code(traceback.format_exc())

# ---------- Visualization (image + safe plot) ----------
st.subheader('Visualization')
safe_image('top_categories.png', caption='Top Categories by Opportunity Score')
st.caption("If the image is missing or outdated, the plot below is generated directly from insights.json:")
try:
    top_cats = pd.DataFrame(ins.get('top_categories', []))
    safe_bar_chart_top_categories(top_cats)
except Exception as e:
    st.error(f"Failed to plot top categories: {e}")
    with st.expander("Details"):
        st.code(traceback.format_exc())

# ---------- Dataset Browser ----------
st.subheader('Browse Combined Dataset')
try:
    safe_dataframe(df.head(200) if not df.empty else df,
                   fallback_msg='combined_apps.csv is empty or missing.')
except Exception as e:
    st.error(f"Failed to display dataset: {e}")
    with st.expander("Details"):
        st.code(traceback.format_exc())

# ---------- Footer ----------
st.markdown('---')
st.caption(f"Generated at: {ins.get('generated_at_utc', 'N/A')}")
"""

with open(os.path.join(OUTPUT_DIR,"app.py"), "w", encoding="utf-8") as f:
    f.write(app_py)

print("Wrote app.py to", OUTPUT_DIR)


Wrote app.py to /kaggle/working


In [51]:
def find_phase5_xlsx() -> Optional[str]:
    for root in PHASE5_XLSX_CANDIDATES:
        if not os.path.exists(root): 
            continue
        if os.path.isfile(root) and root.lower().endswith(".xlsx"):
            return root
        for dirpath, _, filenames in os.walk(root):
            for f in filenames:
                fl = f.lower()
                if fl.endswith(".xlsx") and ("kasparro" in fl or "phase5" in fl or "d2c" in fl):
                    return os.path.join(dirpath, f)
    return None

def load_phase5_excel() -> Dict[str, pd.DataFrame]:
    real_path = find_phase5_xlsx()
    if not real_path:
        print("Phase 5 XLSX not found; skipping.")
        return {}
    print("Using Phase 5 file:", real_path)
    xls = pd.ExcelFile(real_path)
    return {name: pd.read_excel(xls, name) for name in xls.sheet_names}

def compute_phase5_insights(sheets: Dict[str, pd.DataFrame]) -> Dict[str, Any]:
    out = {"found_sheets": list(sheets.keys())}
    def pick(df, name_lc):
        cmap = {c.lower(): c for c in df.columns}
        return cmap.get(name_lc)

    ads_df = funnel_df = seo_df = None
    for _, df in sheets.items():
        cols = {c.lower() for c in df.columns}
        if not ads_df and {"spend","impressions","clicks"} <= cols: ads_df = df.copy()
        if not funnel_df and "installs" in cols and "signups" in cols: funnel_df = df.copy()
        if not seo_df and "search volume" in cols and ("avg position" in cols or "average position" in cols): seo_df = df.copy()

    if ads_df is not None:
        spend_c = pick(ads_df,"spend"); revenue_c = pick(ads_df,"revenue"); conv_c = pick(ads_df,"conversions")
        if conv_c: ads_df["CAC"]  = ads_df[spend_c] / ads_df[conv_c].replace(0, np.nan)
        if revenue_c and spend_c: ads_df["ROAS"] = ads_df[revenue_c] / ads_df[spend_c].replace(0, np.nan)
        out["ad_campaigns_summary"] = ads_df.describe(include="all").to_dict()

    if funnel_df is not None:
        installs_c = pick(funnel_df, "installs"); signups_c = pick(funnel_df, "signups")
        fp_c = pick(funnel_df, "first purchase"); rp_c = pick(funnel_df, "repeat purchase")
        if fp_c and signups_c:
            funnel_df["signup_to_first_purchase"] = funnel_df[fp_c] / funnel_df[signups_c].replace(0, np.nan)
        if rp_c and installs_c:
            funnel_df["repeat_rate_vs_installs"] = funnel_df[rp_c] / funnel_df[installs_c].replace(0, np.nan)
        out["funnel_summary"] = funnel_df.describe(include="all").to_dict()

    if seo_df is not None:
        sv_c = pick(seo_df,"search volume"); pos_c = pick(seo_df,"avg position") or pick(seo_df,"average position")
        conv_c = pick(seo_df,"conversion rate") or pick(seo_df,"conv rate") or pick(seo_df,"cr")
        work = seo_df.copy()
        if not conv_c:
            work["conv_rate_guess"] = np.linspace(0.02, 0.06, len(work))
            conv_c = "conv_rate_guess"
        work["opportunity"] = (
            work[sv_c].rank(pct=True) +
            (1 - work[pos_c].rank(pct=True) if pos_c else 0.5) +
            work[conv_c].rank(pct=True)
        )
        out["seo_opportunities_top"] = work.sort_values("opportunity", ascending=False).head(10).to_dict(orient="records")

    return out

phase5_sheets = load_phase5_excel()
phase5 = compute_phase5_insights(phase5_sheets) if phase5_sheets else {}

def generate_creatives(topic: str) -> Dict[str, str]:
    if not GEMINI_API_KEY:
        return {
            "ad_headline": f"Upgrade Your {topic} — Built to Last.",
            "seo_meta":    f"Discover {topic} with fair pricing and fast delivery.",
            "pdp_text":    f"{topic} designed for performance and comfort. Trusted by thousands."
        }
    try:
        import google.generativeai as genai
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel("models/gemini-1.0-pro")
        prompt = f"""
Create 3 outputs for '{topic}' in JSON with keys: ad_headline, seo_meta, pdp_text.
Constraints:
- ad_headline: < 12 words
- seo_meta: < 155 characters
- pdp_text: ~2 sentences, benefits-focused, trustworthy, non-hype
Only return valid JSON.
"""
        resp = model.generate_content(prompt)
        txt = resp.text or ""
        m = re.search(r"\{.*\}", txt, re.S)
        return json.loads(m.group(0)) if m else {"raw": txt}
    except Exception as e:
        return {
            "ad_headline": f"Level Up Your {topic} Today.",
            "seo_meta":    f"Shop {topic} with quality and value. Free returns.",
            "pdp_text":    f"{topic} built for everyday use with durable materials and clean design.",
            "note": f"Gemini error fallback: {e}"
        }

sample_creatives = generate_creatives("Wireless Earbuds")

phase5_path = os.path.join(OUTPUT_DIR, "phase5_d2c_insights.json")
with open(phase5_path, "w", encoding="utf-8") as f:
    json.dump({
        "generated_at_utc": pd.Timestamp.utcnow().isoformat(),
        "insights": phase5,
        "sample_creatives": sample_creatives
    }, f, indent=2)
phase5_path


Using Phase 5 file: /kaggle/input/kasparro-phase5-d2c-synthetic-dataset/Kasparro_Phase5_D2C_Synthetic_Dataset.xlsx


'/kaggle/working/phase5_d2c_insights.json'